In [22]:
import json
import pandas as pd
import numpy as np

In [276]:
with open('plinf-abdee-export.json') as file:
    data = json.load(file)
results = data['results']

assignemnts_data = pd.read_csv("Batch_4270661_batch_results.csv", header=0)
assignemnts_data_2 = pd.read_csv("Batch_4270670_batch_results.csv", header=0)
id_list = np.append(assignemnts_data['Answer.experimentcode'].values, assignemnts_data_2['Answer.experimentcode'].values)
id_list = list(map(str, id_list))
workers_dict = dict.fromkeys(id_list, 0)
exp_dict = dict()
possibly_reject = []

In [277]:
ids_dict = dict.fromkeys(id_list, 0)
for key, entry in results.items():
    key = key.split("_")
    current_id = key[0]
    if current_id not in ids_dict.keys(): 
        continue
    if ids_dict[current_id] == 0: 
        ids_dict[current_id] = {}
    if len(key) == 2: #the entry is tutorial
        ids_dict[current_id]['tutorial'] = entry
    else:
        exp = "_".join(key[-2:])
        ids_dict[current_id][exp] = entry
        

In [278]:
workers_dict = dict.fromkeys(id_list, 0)
exp_dict = dict()
for key, entry in results.items():
    key = key.split("_")
    current_id = key[0]
    if current_id not in workers_dict.keys(): 
        continue 
    if workers_dict[current_id] == 0: 
        workers_dict[current_id] = {'reward_score': 0, 'min_time': 1000000, 'default': False}
    if len(key) == 2: #the entry is tutorial
        workers_dict[current_id]['tutorial_final_prob'] = entry[-1]['true_goal_probs']  
    else: #the entry is an experiment
        exp = "_".join(key[-2:])
        if exp in exp_dict.keys(): 
            exp_dict[exp].append(current_id)
        else: 
            exp_dict[exp] = [current_id]
        workers_dict[current_id]['reward_score'] += entry[-1]['reward_score']/entry[-1]['timestep']
        time = []
        for step in entry: 
            time.append(step['time_spent'])
        if min(time) < workers_dict[current_id]['min_time'] and min(time) != 0: 
            workers_dict[current_id]['min_time'] = min(time)
        if [entry[-1]['goal_probs_0'], entry[-1]['goal_probs_1'], 
            entry[-1]['goal_probs_2'], entry[-1]['goal_probs_3'], 
            entry[-1]['goal_probs_4']] == [0.2,0.2,0.2,0.2,0.2]: 
            workers_dict[current_id]['default'] = True

In [279]:
#flaggining workers 
flagged_ids = []
for key, worker in workers_dict.items(): 
    if worker['min_time'] < 2.0 or worker['tutorial_final_prob'] < 0.7 or worker['default']: 
        worker['flag'] = True
        flagged_ids.append(key)
    else: 
        worker['flag'] = False
        

In [280]:
#counting experiments
exp_counter = {}
for key, exp in exp_dict.items(): 
    exp_counter[key] = {}
    exp_counter[key]['total'] = len(exp)
    intersection = list(set(exp) & set(flagged_ids)) 
    exp_counter[key]['possibly_rejected'] = len(intersection)
    
    

In [281]:
exp_df = pd.DataFrame.from_dict(exp_counter, orient='index', dtype=None, columns=None)
exp_df

,total,possibly_rejected
1_1,10,5
2_1,10,5
3_4,19,12
0_1,9,4
1_3,9,4
4_3,9,4
0_3,12,6
2_2,12,6
4_1,12,6
1_2,9,4


In [282]:
workers_df = pd.DataFrame.from_dict(workers_dict, orient='index', dtype=None, columns=None)
workers_df

,reward_score,min_time,default,tutorial_final_prob,flag
1607118747929,1.222944,4.096,False,1.000000,False
1607116558958,1.210433,3.808,False,0.555556,True
1607116125693,0.872030,4.096,False,0.666667,True
1607115201867,1.498016,3.732,False,0.714286,False
1607125963924,0.566166,6.640,False,0.189189,True
...,...,...,...,...,...
1607117172687,0.622174,0.546,False,0.200000,True
1607115360280,0.858327,4.424,False,0.200000,True
1607116510666,0.550868,2.896,False,0.238095,True
1607115486517,1.540000,3.536,False,0.500000,True


In [283]:
ids_dict[flagged_ids[0]]

{'1_4': [{'goal_probs_0': 0.3333333333333333,
   'goal_probs_1': 0.5333333333333333,
   'goal_probs_2': 0,
   'goal_probs_3': 0.06666666666666667,
   'goal_probs_4': 0.06666666666666667,
   'reward_score': 0.3333333333333333,
   'time_spent': 0,
   'timestep': 1,
   'true_goal_probs': 0.3333333333333333},
  {'goal_probs_0': 0.47368421052631576,
   'goal_probs_1': 0.42105263157894735,
   'goal_probs_2': 0,
   'goal_probs_3': 0.05263157894736842,
   'goal_probs_4': 0.05263157894736842,
   'reward_score': 0.8070175438596491,
   'time_spent': 0,
   'timestep': 2,
   'true_goal_probs': 0.47368421052631576},
  {'goal_probs_0': 0.47368421052631576,
   'goal_probs_1': 0.42105263157894735,
   'goal_probs_2': 0,
   'goal_probs_3': 0.05263157894736842,
   'goal_probs_4': 0.05263157894736842,
   'reward_score': 1.280701754385965,
   'time_spent': 0,
   'timestep': 3,
   'true_goal_probs': 0.47368421052631576},
  {'goal_probs_0': 0.47368421052631576,
   'goal_probs_1': 0.42105263157894735,
   'goal

In [284]:
with open('dict_by_id.json', 'w') as fp:
    json.dump(ids_dict, fp)

In [288]:
exp_df.to_csv(path_or_buf='experiments_data.csv')
